In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from importlib import reload

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from sklearn.metrics import roc_auc_score
from sklearn import mixture, preprocessing, datasets

from importlib import reload
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import patches

import torch.utils.data as data_utils

import utils.mahalanobis as resnet

import utils.dataloaders as dl
import tqdm

import model_params as params

In [88]:
device = torch.device('cuda:1')

pre_trained_net = '../deep_Mahalanobis_detector/pre_trained/resnet_cifar10.pth'

In [89]:
model = resnet.ResNet34(num_c=10)
model.load_state_dict(torch.load(pre_trained_net, map_location = device))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [90]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [5]:
in_loader = dl.CIFAR10(train=False)

In [7]:
def compute_layer_statistics(model, device, loader):
    model.eval()
    classes = 10
    class_num = ((torch.tensor(loader.dataset.targets)[:,None]
                  == torch.arange(10)[None,:]).sum(0).float())
    
    with torch.no_grad():
    
        data = enumerate(loader).__next__()[1][0].to(device)
        _, act = model.penultimate_forward(data)
        act = [act]

        dims = [a[0].view(-1).shape[0] for a in act]
        mean = [torch.zeros(classes, d) for d in dims]
        cov = [torch.zeros(d, d) for d in dims]

        for data, label in loader:
            data, label = data.to(device), label.to(device)
            label = torch.zeros(label.shape[0], classes, 
                                device=device).scatter_(1, label[:,None], 1)

            _, act = model.penultimate_forward(data)

            for i, layer_act in enumerate([act]):
                mean[i] += (layer_act.view(layer_act.shape[0], -1)[:,None,:]
                            * label[:,:,None]).sum(0).cpu()
        for i, _ in enumerate(mean):
            mean[i] = mean[i] / class_num[:,None]
        
        for idx, (data, label) in enumerate(loader):
            print(idx)
            data = data.to(device)
            label = label.to(device)
            
            _, act = model.penultimate_forward(data)

            for i, layer_act in enumerate([act]):
                diff = (layer_act.view(layer_act.shape[0], -1).cpu() - mean[i][label])
                cov[i] += (diff[:,:,None] * diff[:,None,:]).sum(0)
                
        for i, _ in enumerate(cov):
            cov[i] = cov[i] / class_num.sum()
        return mean, cov

In [8]:
mean, cov = compute_layer_statistics(model, device, in_loader)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [79]:
reload(resnet)

<module 'utils.mahalanobis' from '/home/alexm/project/notebooks/gmm-robust/utils/mahalanobis.py'>

In [91]:
odin = resnet.ModelODIN(model, 0.0004, mean[0], cov[0], device)

In [82]:
odin, T, eps = resnet.grid_search_variables(model, model_params, device)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [64]:
x = next(iter(in_loader))[0].to(device)

In [16]:
model_params = params.params_dict['CIFAR10']()

In [81]:
odin(x)

tensor([[ -31.0850,  -16.9696,  -16.4578,   -5.0314,  -18.1555,  -13.7981,
          -21.4746,  -32.3519,  -22.7567,  -35.2783],
        [ -33.1156,  -33.4961,  -41.5007,  -37.3031,  -43.9631,  -40.6855,
          -42.6369,  -48.6889,   -7.2270,  -45.9021],
        [ -68.2058,  -18.0579,  -70.7925,  -46.7569,  -72.9105,  -65.1016,
          -75.1489,  -89.0507,  -57.3310,  -38.0976],
        [  -9.6770,  -36.2028,  -50.5123,  -47.4465,  -58.2253,  -30.5233,
          -51.2334,  -58.9126,  -25.3214,  -51.5401],
        [ -84.0681,  -74.0115,  -56.7595,  -73.8843,  -67.7269,  -65.8684,
          -23.7110,  -91.7916,  -76.1788,  -73.0908],
        [ -29.4991,  -30.7605,  -23.4460,   -9.4966,  -27.3751,  -16.1693,
          -29.0386,  -10.4514,  -38.9258,  -26.3791],
        [ -66.9303,  -12.5126,  -64.9145,  -59.6089,  -58.5747,  -55.9651,
          -63.9920,  -84.1561,  -62.8276,  -54.6009],
        [ -32.4723,  -44.2659,  -15.4199,  -26.4733,   -6.4765,  -24.8320,
          -26.9652,  -

In [85]:
odin.epsilon

0.0004

In [67]:
out = odin(x)

In [92]:
odin.cpu()
torch.save(odin, 'SavedModels/other/' + 'mahalanobis_CIFAR10' + '.pth')

In [66]:
x.shape

torch.Size([100, 3, 32, 32])

In [87]:
reload(resnet)

<module 'utils.mahalanobis' from '/home/alexm/project/notebooks/gmm-robust/utils/mahalanobis.py'>